<a href="https://colab.research.google.com/github/YashShekhar/Generative-AI/blob/main/Build_a_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build a Chatbot

In [1]:
!pip install -U langchain-groq

In [2]:
!pip install langchain-core langgraph>0.2.27

In [3]:
## Setup API KEY - Google Colab
import os
from google.colab import userdata
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')

In [4]:
from langchain.chat_models import init_chat_model

model = init_chat_model(
    model="openai/gpt-oss-120b",
    model_provider="groq"
)

In [5]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi! I am Yash")])

AIMessage(content='Hello Yash! 👋 Nice to meet you. How can I help you today?', additional_kwargs={'reasoning_content': 'The user says "Hi! I am Yash". Likely a greeting, maybe wants to respond. Should respond friendly. No special instructions.'}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 77, 'total_tokens': 134, 'completion_time': 0.115624988, 'prompt_time': 0.003754858, 'queue_time': 0.216534622, 'total_time': 0.119379846}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_213abb2467', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--6996cecf-f420-49f4-a955-e7d6a6a71c3f-0', usage_metadata={'input_tokens': 77, 'output_tokens': 57, 'total_tokens': 134})

In [11]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content='I’m not able to see your name. If you’d like me to address you personally, just let me know what you’d prefer to be called!', additional_kwargs={'reasoning_content': 'The user asks "What\'s my name?" The assistant doesn\'t have any prior info about the user. According to policy, we must respond that we don\'t know. We can ask politely for clarification.'}, response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 75, 'total_tokens': 154, 'completion_time': 0.156947468, 'prompt_time': 0.003092402, 'queue_time': 0.212843506, 'total_time': 0.16003987}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_8db49de948', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--5f2c8406-fef1-4eaf-be11-272d05c5fc10-0', usage_metadata={'input_tokens': 75, 'output_tokens': 79, 'total_tokens': 154})

In [12]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I am Yash"),
        AIMessage(content='Hello Yash! 👋 How can I help you today?'),
        HumanMessage(content="What's my name?")
    ]
)

AIMessage(content='Your name is Yash. 😊', additional_kwargs={'reasoning_content': 'The user asks "What\'s my name?" We have context: earlier they said "Hi! I am Yash". So name is Yash. Should answer accordingly.'}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 104, 'total_tokens': 154, 'completion_time': 0.10686029, 'prompt_time': 0.004861757, 'queue_time': 0.407175644, 'total_time': 0.111722047}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_82669fd91d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--82895186-caa3-402e-9e01-c7b2afabfc45-0', usage_metadata={'input_tokens': 104, 'output_tokens': 50, 'total_tokens': 154})

## Message Persistence

In [6]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

In [7]:
# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

# Define the function that calls the model
def call_model(state: MessagesState):
  response = model.invoke(state["messages"])
  return {"messages": response}

# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [8]:
config = {"configurable": {"thread_id": "abc123"}}

In [10]:
query = "Hi! I'm Yash."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output

{'messages': [HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}, id='2ef7dfe9-8a1c-4f77-86f6-ba1dc62078b6'),
  AIMessage(content='I don’t have any information about your name. If you’d like, feel free to tell me what you’d like to be called, and I’ll use that!', additional_kwargs={'reasoning_content': 'The user asks "What\'s my name?" There\'s no prior context. According to policy, we must not fabricate personal info. We should respond that we don\'t have that info. Apologize and ask if they\'d like to share.'}, response_metadata={'token_usage': {'completion_tokens': 89, 'prompt_tokens': 75, 'total_tokens': 164, 'completion_time': 0.188806465, 'prompt_time': 0.003646782, 'queue_time': 0.53532412, 'total_time': 0.192453247}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_8db49de948', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--27406027-b41e-482e-abd2-0e4015066401-0', usage_metadata={'input_tokens': 75

In [12]:
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Nice to meet you, Yash! How can I help you today?


In [11]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Yash.


In [12]:
# If we change the config i.e., thread_id. It will start a new conversation

config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I don’t have any information about your name. If you’d like to share it, feel free to let me know!


In [13]:
config = {"configurable": {"thread_id": "abc123"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Yash.
